<a href="https://colab.research.google.com/github/CcTheresa/Notia/blob/main/emotion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook focuses on finetuning the DistilBERT model from Hugging Face to classify emotions in text.

I used the Go emotions dataset as its multilabel aspect is ideal for journal entires which often reflect a myriad of emtions.

In [1]:
#installing the necessary libraries
!pip install transformers datasets evaluate

#Import the necessary dependancies
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer #tokenizer & model, training args and trainer
from datasets import load_dataset, DatasetDict  #accessing the dataset
import evaluate  #evaluation metrics
import numpy as np

#Load dataset
ds = load_dataset("google-research-datasets/go_emotions", "simplified")

#Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=27)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

#Tokenizing the dataset:to ensure consistent input format to model
# The tokenizer converts raw text into token IDs
# Padding ensures all sequences are the same length
# Truncation cuts off very long sequences so they fit the model
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

# Apply the tokenizer across the entire dataset
# batched=True = process multiple examples at once for speed
tokenized_dataset = ds.map(tokenize, batched=True)

# Check one sample after tokenization
print(tokenized_dataset["train"][0])


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

{'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'id': 'eebbqej', 'input_ids': [101, 2026, 8837, 2833, 2003, 2505, 1045, 2134, 1005, 1056, 2031, 2000, 5660, 2870, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [2]:
# Load metrics
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(eval_pred):
    logits, labels = eval_pred  # model output vs. true labels
    probs = sigmoid(logits)     # convert logits to probabilities (0 → 1 scale)
    preds = (probs >= 0.5).astype(int)   # threshold at 0.5 → predict 1 if probability ≥ 0.5

    results = {
        # Micro: aggregate over all predictions
        "precision_micro": precision_metric.compute(predictions=preds, references=labels, average="micro")["precision"],
        "recall_micro": recall_metric.compute(predictions=preds, references=labels, average="micro")["recall"],
        "f1_micro": f1_metric.compute(predictions=preds, references=labels, average="micro")["f1"],

        # Macro: average per class (ignores frequency)
        "precision_macro": precision_metric.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall_macro": recall_metric.compute(predictions=preds, references=labels, average="macro")["recall"],
        "f1_macro": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
    }
    return results


In [3]:
# TRAINING BLOCK (corrected)
# Assumes: `ds` (datasets.DatasetDict) and `tokenizer` are already in the notebook workspace.

# Imports
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
import evaluate
import numpy as np
import torch
import torch.nn as nn

# (2) constants
NUM_LABELS = 27
MAX_LEN = 128

# (3) tokenizer mapping (tokenize texts)
def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

# Dataset mapping for multilabel flattening
def to_multilabel_flat(batch):

    out_labels = []
    for labs in batch["labels"]:
        # handle numpy / torch / nested single-element wrappers
        if isinstance(labs, (np.ndarray, torch.Tensor)):
            labs = labs.tolist()

        # if nested wrapper like [[...]] -> unwrap
        if isinstance(labs, list) and len(labs) == 1 and isinstance(labs[0], (list, np.ndarray, torch.Tensor)):
            labs = labs[0]
            if isinstance(labs, (np.ndarray, torch.Tensor)):
                labs = labs.tolist()

        # Case 1: list of integer indices -> build multi-hot vector
        if isinstance(labs, list) and all(isinstance(x, int) for x in labs):
            row = [0.0] * NUM_LABELS
            for idx in labs:
                if 0 <= idx < NUM_LABELS:
                    row[idx] = 1.0
            out_labels.append(row)
            continue

        # Case 2: already a sequence of 0/1 values or floats (maybe wrong length) -> normalize
        if isinstance(labs, list) and all(isinstance(x, (int, float)) for x in labs):
            # convert all to float
            arr = [float(x) for x in labs]
            # pad or truncate to NUM_LABELS
            if len(arr) < NUM_LABELS:
                arr = arr + [0.0] * (NUM_LABELS - len(arr))
            elif len(arr) > NUM_LABELS:
                arr = arr[:NUM_LABELS]
            out_labels.append(arr)
            continue

        # Fallback: unexpected shape/types -> create zero vector (safe)
        out_labels.append([0.0] * NUM_LABELS)

    batch["labels"] = out_labels
    return batch

tokenized = ds.map(tokenize_fn, batched=True)
tokenized = tokenized.map(to_multilabel_flat, batched=True)

# (C) finally set HF dataset format so Trainer gets tensors
# Do NOT pass dtype as dict (that caused earlier TypeError). If labels are floats (Python lists), HF will convert to torch.float.
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])




# Sanity checks (run these and confirm)
example = tokenized["train"][0]
print("keys:", example.keys())
print("input_ids dtype:", example["input_ids"].dtype, "shape:", example["input_ids"].shape)
print("attention_mask dtype:", example["attention_mask"].dtype, "shape:", example["attention_mask"].shape)
print("labels dtype:", example["labels"].dtype, "shape:", example["labels"].shape)  # should be torch.float32 and (27,)
print("labels sample:", example["labels"])

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

keys: dict_keys(['labels', 'input_ids', 'attention_mask'])
input_ids dtype: torch.int64 shape: torch.Size([128])
attention_mask dtype: torch.int64 shape: torch.Size([128])
labels dtype: torch.int64 shape: torch.Size([27])
labels sample: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])


In [ ]:

# (6) Create train/validation subsets (optional quick subsets)
train_subset = tokenized["train"].shuffle(seed=42).select(range(5000))
eval_subset  = tokenized["validation"].shuffle(seed=42).select(range(1000))

# (7) build model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=NUM_LABELS
)

# (8) metrics + compute_metrics (multi-label via sigmoid + threshold)
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(eval_pred):
    logits, labels = eval_pred   # logits: (batch_size, NUM_LABELS), labels: (batch_size, NUM_LABELS)
    probs = sigmoid(logits)
    preds = (probs >= 0.5).astype(int)

    results = {
        # micro
        "precision_micro": precision_metric.compute(predictions=preds, references=labels, average="micro")["precision"],
        "recall_micro":    recall_metric.compute(predictions=preds, references=labels, average="micro")["recall"],
        "f1_micro":        f1_metric.compute(predictions=preds, references=labels, average="micro")["f1"],
        # macro
        "precision_macro": precision_metric.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall_macro":    recall_metric.compute(predictions=preds, references=labels, average="macro")["recall"],
        "f1_macro":        f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
    }
    return results

# (9) data collator
data_collator = DataCollatorWithPadding(tokenizer)

# (10) training args (disable wandb with report_to="none")
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",   # correct param name
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    logging_steps=100,
    seed=42,
    report_to="none"   # disable wandb logging / API prompt
)
# Custom Trainer to fix label dtype in loss computation
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Convert labels to float here to work with BCEWithLogitsLoss
        labels = inputs.pop("labels") # pop labels from inputs so model doesn't compute loss internally
        labels = labels.float()       # convert labels to float for BCEWithLogitsLoss


        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1, self.model.config.num_labels))

        return (loss, outputs) if return_outputs else loss

# (11) create trainer (use processing_class to avoid deprecation warning)
trainer = MultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    tokenizer=tokenizer,   # avoids the tokenizer deprecation warning
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# (12) train
trainer.train()

# (13) save
trainer.save_model("./finetuned_distilbert_goemotions")
tokenizer.save_pretrained("./finetuned_distilbert_goemotions")
print("Training finished. Model saved to ./finetuned_distilbert_goemotions")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2993506725.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultilabelTrainer.__init__`. Use `processing_class` instead.
  trainer = MultilabelTrainer(


Epoch,Training Loss,Validation Loss
